In [14]:
# Import Libraries
import pandas as pd
import numpy as np

##### **1. Load and Merge Datasets**

***Purpose: Load the datasets (static_data, vitals_data, clinical_codes), aggregate vitals_data to the ICU stay level by computing means, and merge all datasets into merged_data***

In [15]:
# Load and Merge Datasets
# Load the datasets
static_data = pd.read_csv('C:/MIMIC-Extract/output/static_data.csv')
vitals_data = pd.read_hdf('C:/MIMIC-Extract/output/vitals_hourly_data.h5')
clinical_codes = pd.read_hdf('C:/MIMIC-Extract/output/C.h5')

# Step 1a: Aggregate vitals_hourly_data.h5 to the ICU stay level
# Compute the mean of each clinical variable (LEVEL2) over the time series
vitals_agg = vitals_data.groupby('icustay_id').agg({
    (col, 'mean'): 'mean' for col in vitals_data.columns.levels[0] if col != 'hours_in'
})
# Flatten the MultiIndex columns
vitals_agg.columns = [f"{col[0]}_mean" for col in vitals_agg.columns]
vitals_agg = vitals_agg.reset_index()

# Step 1b: Prepare static_data and clinical_codes
static_data = static_data.set_index('icustay_id')

# Drop redundant columns from clinical_codes after setting the index
clinical_codes = clinical_codes.reset_index().set_index('icustay_id')
clinical_codes = clinical_codes.drop(columns=['subject_id', 'hadm_id'], errors='ignore')

# Step 1c: Merge the datasets
merged_data = static_data.join(vitals_agg.set_index('icustay_id'), how='left')
merged_data = merged_data.join(clinical_codes, how='left')

# Reset index to make icustay_id a column
merged_data = merged_data.reset_index()

# Print dataset details
print("Shape of merged dataset:", merged_data.shape)
print("Columns in merged dataset:", merged_data.columns.tolist())

Shape of merged dataset: (34472, 136)
Columns in merged dataset: ['icustay_id', 'subject_id', 'hadm_id', 'gender', 'ethnicity', 'age', 'insurance', 'admittime', 'diagnosis_at_admission', 'dischtime', 'discharge_location', 'fullcode_first', 'dnr_first', 'fullcode', 'dnr', 'dnr_first_charttime', 'timecmo_chart', 'cmo_first', 'cmo_last', 'cmo', 'deathtime', 'intime', 'outtime', 'los_icu', 'admission_type', 'first_careunit', 'mort_icu', 'mort_hosp', 'hospital_expire_flag', 'hospstay_seq', 'readmission_30', 'Alanine aminotransferase_mean', 'Albumin_mean', 'Albumin ascites_mean', 'Albumin pleural_mean', 'Albumin urine_mean', 'Alkaline phosphate_mean', 'Anion gap_mean', 'Asparate aminotransferase_mean', 'Basophils_mean', 'Bicarbonate_mean', 'Bilirubin_mean', 'Blood urea nitrogen_mean', 'CO2_mean', 'CO2 (ETCO2, PCO2, etc.)_mean', 'Calcium_mean', 'Calcium ionized_mean', 'Calcium urine_mean', 'Cardiac Index_mean', 'Cardiac Output Thermodilution_mean', 'Cardiac Output fick_mean', 'Central Venous 

#### ***2. Load MIMIC-III Tables***

In [16]:
# Load MIMIC-III Tables
inputevents_cv = pd.read_csv('C:/MIMIC-III/mimic-iii-clinical-database-1.4/data/INPUTEVENTS_CV.csv')
inputevents_mv = pd.read_csv('C:/MIMIC-III/mimic-iii-clinical-database-1.4/data/INPUTEVENTS_MV.csv')
d_items = pd.read_csv('C:/MIMIC-III/mimic-iii-clinical-database-1.4/data/D_ITEMS.csv')

C:\Users\KhanhChang\AppData\Local\Temp\ipykernel_25240\1388829671.py:2: DtypeWarning: Columns (7,9,17,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  inputevents_cv = pd.read_csv('C:/MIMIC-III/mimic-iii-clinical-database-1.4/data/INPUTEVENTS_CV.csv')


#### ***3. Identify Interventions in D_ITEMS***

***Purpose: Filter the D_ITEMS table to identify ITEMIDs for vasopressors, antibiotics, and sedatives, and confirm their labels and categories.***

In [17]:
# Identify Interventions in D_ITEMS
# Filter D_ITEMS for relevant categories (Medications, Antibiotics, Sedatives, Vasopressors)
relevant_categories = ['Medications', 'Antibiotics', 'Sedatives', 'Vasopressors']
medication_items = d_items[d_items['CATEGORY'].str.contains('|'.join(relevant_categories), na=False, case=False)]

# Identify ITEMIDs for common interventions
vasopressors = medication_items[medication_items['LABEL'].str.contains('Norepinephrine|Dopamine|Epinephrine', case=False, na=False)]
antibiotics = medication_items[medication_items['LABEL'].str.contains('Vancomycin|Piperacillin|Ceftriaxone', case=False, na=False)]
sedatives = medication_items[medication_items['LABEL'].str.contains('Propofol|Midazolam', case=False, na=False)]

# Create lists of ITEMIDs
vasopressor_itemids = vasopressors['ITEMID'].tolist()
antibiotic_itemids = antibiotics['ITEMID'].tolist()
sedative_itemids = sedatives['ITEMID'].tolist()

# Print the ITEMID lists
print("Vasopressor ITEMIDs:", vasopressor_itemids)
print("Antibiotic ITEMIDs:", antibiotic_itemids)
print("Sedative ITEMIDs:", sedative_itemids)

# Confirm labels for the identified ITEMIDs
print("\nVasopressors:")
print(medication_items[medication_items['ITEMID'].isin(vasopressor_itemids)][['ITEMID', 'LABEL', 'CATEGORY']])
print("\nAntibiotics:")
print(medication_items[medication_items['ITEMID'].isin(antibiotic_itemids)][['ITEMID', 'LABEL', 'CATEGORY']])
print("\nSedatives:")
print(medication_items[medication_items['ITEMID'].isin(sedative_itemids)][['ITEMID', 'LABEL', 'CATEGORY']])

Vasopressor ITEMIDs: [221289, 221662, 221906]
Antibiotic ITEMIDs: [225855, 225892, 225893, 225798]
Sedative ITEMIDs: [221668, 222168]

Vasopressors:
      ITEMID           LABEL     CATEGORY
8879  221289     Epinephrine  Medications
8891  221662        Dopamine  Medications
9553  221906  Norepinephrine  Medications

Antibiotics:
       ITEMID                            LABEL     CATEGORY
9863   225855                      Ceftriaxone  Antibiotics
9888   225892                     Piperacillin  Antibiotics
9889   225893  Piperacillin/Tazobactam (Zosyn)  Antibiotics
10218  225798                       Vancomycin  Antibiotics

Sedatives:
      ITEMID               LABEL     CATEGORY
8892  221668  Midazolam (Versed)  Medications
9564  222168            Propofol  Medications


#### ***4. Analyze all interventions in Cohort***

***Purpose: Analyze all interventions in your cohort by extracting unique ITEMIDs from INPUTEVENTS_CV and INPUTEVENTS_MV, mapping them to labels, and calculating the number and proportion of patients who received each intervention.***

In [18]:
# Analyze All Interventions in Cohort
# Extract unique ITEMIDs from both INPUTEVENTS tables
cv_itemids = inputevents_cv['ITEMID'].unique()
mv_itemids = inputevents_mv['ITEMID'].unique()
all_itemids = set(cv_itemids).union(set(mv_itemids))

# Map ITEMIDs to labels using D_ITEMS
itemid_to_label = d_items[d_items['ITEMID'].isin(all_itemids)][['ITEMID', 'LABEL']].set_index('ITEMID')['LABEL'].to_dict()

# Print all interventions (ITEMID and LABEL)
print("All Interventions Available in INPUTEVENTS_CV and INPUTEVENTS_MV:")
for itemid in sorted(all_itemids):
    label = itemid_to_label.get(itemid, "Unknown")
    print(f"ITEMID: {itemid}, Label: {label}")

# Get unique icustay_ids from merged_data
icustay_ids = merged_data['icustay_id'].unique()

# Filter INPUTEVENTS_CV and INPUTEVENTS_MV to only include your cohort
inputevents_cv = inputevents_cv[inputevents_cv['ICUSTAY_ID'].isin(icustay_ids)]
inputevents_mv = inputevents_mv[inputevents_mv['ICUSTAY_ID'].isin(icustay_ids)]

# Combine CV and MV data
inputevents = pd.concat([
    inputevents_cv[['ICUSTAY_ID', 'ITEMID']],
    inputevents_mv[['ICUSTAY_ID', 'ITEMID']]
])

# Group by ICUSTAY_ID and ITEMID to identify which patients received each intervention
intervention_counts = inputevents.groupby(['ITEMID', 'ICUSTAY_ID']).size().reset_index(name='count')
intervention_presence = intervention_counts.groupby('ITEMID')['ICUSTAY_ID'].nunique().reset_index(name='num_patients')

# Add labels from D_ITEMS
intervention_presence = intervention_presence.merge(
    d_items[['ITEMID', 'LABEL']],
    on='ITEMID',
    how='left'
)

# Calculate the proportion of patients who received each intervention
total_patients = len(icustay_ids)  # 34,472
intervention_presence['proportion'] = intervention_presence['num_patients'] / total_patients

# Sort by number of patients (descending)
intervention_presence = intervention_presence.sort_values(by='num_patients', ascending=False)

# Print the summary
print(f"\nTotal ICU Stays in Cohort: {total_patients}")
print("\nInterventions in Your Cohort (Sorted by Number of Patients):")
print(intervention_presence[['ITEMID', 'LABEL', 'num_patients', 'proportion']].head(10))

All Interventions Available in INPUTEVENTS_CV and INPUTEVENTS_MV:
ITEMID: 30001, Label: Packed RBC's
ITEMID: 30002, Label: Whole Blood
ITEMID: 30003, Label: Aminophylline
ITEMID: 30004, Label: Washed PRBC's
ITEMID: 30005, Label: Fresh Frozen Plasma
ITEMID: 30006, Label: Platelets
ITEMID: 30007, Label: Cryoprecipitate
ITEMID: 30008, Label: Albumin 5%
ITEMID: 30009, Label: Albumin 25%
ITEMID: 30011, Label: Dextran 40
ITEMID: 30012, Label: Hespan
ITEMID: 30013, Label: D5W
ITEMID: 30014, Label: D5/.2NS
ITEMID: 30015, Label: D5/.45NS
ITEMID: 30016, Label: Dextrose 10%
ITEMID: 30017, Label: Dextrose 20%
ITEMID: 30018, Label: .9% Normal Saline
ITEMID: 30020, Label: .45% Normal Saline
ITEMID: 30021, Label: Lactated Ringers
ITEMID: 30022, Label: Calcium Chloride
ITEMID: 30023, Label: Calcium Gluconate
ITEMID: 30024, Label: Ferrous Sulfate
ITEMID: 30025, Label: Heparin
ITEMID: 30026, Label: KCL
ITEMID: 30027, Label: Magnesium Sulfate
ITEMID: 30028, Label: MVI
ITEMID: 30029, Label: Potassium Phos

#### ***5. Update ITEMID Lists and Create Intervention Indicators***

***Purpose: Update the ITEMID lists for vasopressors, antibiotics, and sedatives, and create binary indicators (has_vasopressor, has_antibiotic, has_sedative) for each intervention type.***

In [19]:
# Update ITEMID Lists and Create Intervention Indicators
# Updated ITEMID lists for each intervention type (based on previous analysis)
vasopressor_itemids = [221289, 221662, 221906, 221749, 222315]  # Epinephrine, Dopamine, Norepinephrine, Phenylephrine, Vasopressin
antibiotic_itemids = [225855, 225892, 225893, 225798]  # Ceftriaxone, Piperacillin, Piperacillin/Tazobactam, Vancomycin
sedative_itemids = [221668, 222168, 30131]  # Midazolam, Propofol (MV), Propofol (CV)

print("Updated Vasopressor ITEMIDs:", vasopressor_itemids)
print("Updated Antibiotic ITEMIDs:", antibiotic_itemids)
print("Updated Sedative ITEMIDs:", sedative_itemids)

# Ensure ICUSTAY_ID is integer and not null in both INPUTEVENTS_CV and INPUTEVENTS_MV
inputevents_cv = inputevents_cv[inputevents_cv['ICUSTAY_ID'].notnull()]
inputevents_cv['ICUSTAY_ID'] = inputevents_cv['ICUSTAY_ID'].astype(int)

inputevents_mv = inputevents_mv[inputevents_mv['ICUSTAY_ID'].notnull()]
inputevents_mv['ICUSTAY_ID'] = inputevents_mv['ICUSTAY_ID'].astype(int)

# Initialize the interventions DataFrame with icustay_id
interventions = merged_data[['icustay_id']].copy()

# 1. Vasopressor Indicator
cv_vasopressor = inputevents_cv[inputevents_cv['ITEMID'].isin(vasopressor_itemids)].groupby('ICUSTAY_ID').size().reset_index(name='vasopressor_events')
mv_vasopressor = inputevents_mv[inputevents_mv['ITEMID'].isin(vasopressor_itemids)].groupby('ICUSTAY_ID').size().reset_index(name='vasopressor_events')
vasopressor = pd.concat([cv_vasopressor, mv_vasopressor]).groupby('ICUSTAY_ID')['vasopressor_events'].sum().reset_index()
interventions = interventions.merge(vasopressor, left_on='icustay_id', right_on='ICUSTAY_ID', how='left')
interventions['has_vasopressor'] = (interventions['vasopressor_events'].fillna(0) > 0).astype(int)

# 2. Antibiotic Indicator
cv_antibiotic = inputevents_cv[inputevents_cv['ITEMID'].isin(antibiotic_itemids)].groupby('ICUSTAY_ID').size().reset_index(name='antibiotic_events')
mv_antibiotic = inputevents_mv[inputevents_mv['ITEMID'].isin(antibiotic_itemids)].groupby('ICUSTAY_ID').size().reset_index(name='antibiotic_events')
antibiotic = pd.concat([cv_antibiotic, mv_antibiotic]).groupby('ICUSTAY_ID')['antibiotic_events'].sum().reset_index()
interventions = interventions.merge(antibiotic, left_on='icustay_id', right_on='ICUSTAY_ID', how='left')
interventions['has_antibiotic'] = (interventions['antibiotic_events'].fillna(0) > 0).astype(int)

# 3. Sedative Indicator
cv_sedative = inputevents_cv[inputevents_cv['ITEMID'].isin(sedative_itemids)].groupby('ICUSTAY_ID').size().reset_index(name='sedative_events')
mv_sedative = inputevents_mv[inputevents_mv['ITEMID'].isin(sedative_itemids)].groupby('ICUSTAY_ID').size().reset_index(name='sedative_events')
sedative = pd.concat([cv_sedative, mv_sedative]).groupby('ICUSTAY_ID')['sedative_events'].sum().reset_index()
interventions = interventions.merge(sedative, left_on='icustay_id', right_on='ICUSTAY_ID', how='left')
interventions['has_sedative'] = (interventions['sedative_events'].fillna(0) > 0).astype(int)

# Clean up interventions DataFrame
interventions = interventions.drop(columns=['ICUSTAY_ID_x', 'ICUSTAY_ID_y', 'ICUSTAY_ID', 'vasopressor_events', 'antibiotic_events', 'sedative_events'], errors='ignore')

# Print the interventions DataFrame
print("\nInterventions DataFrame:")
print(interventions.head())

Updated Vasopressor ITEMIDs: [221289, 221662, 221906, 221749, 222315]
Updated Antibiotic ITEMIDs: [225855, 225892, 225893, 225798]
Updated Sedative ITEMIDs: [221668, 222168, 30131]

Interventions DataFrame:
   icustay_id  has_vasopressor  has_antibiotic  has_sedative
0      211552                0               0             0
1      294638                0               0             0
2      228232                0               0             0
3      220597                0               0             1
4      229441                0               0             0


#### ***Define final_data and add intervention indicators***

***Purpose: Define final_data as a copy of merged_data, merge the intervention indicators, and verify the new columns and their distributions.***

In [20]:
# Define final_data as merged_data (if not already defined)
final_data = merged_data.copy()

# Update final_data with the new indicators
final_data = final_data.drop(columns=['has_vasopressor', 'has_antibiotic', 'has_sedative'], errors='ignore')
final_data = final_data.merge(interventions[['icustay_id', 'has_vasopressor', 'has_antibiotic', 'has_sedative']], on='icustay_id', how='left')

# Verify distributions
print("Distributions of Updated Interventions:")
for intervention in ['has_vasopressor', 'has_antibiotic', 'has_sedative']:
    print(f"\n{intervention}:")
    print(final_data[intervention].value_counts(normalize=True))

Distributions of Updated Interventions:

has_vasopressor:
has_vasopressor
0    0.879148
1    0.120852
Name: proportion, dtype: float64

has_antibiotic:
has_antibiotic
0    0.837433
1    0.162567
Name: proportion, dtype: float64

has_sedative:
has_sedative
0    0.593525
1    0.406475
Name: proportion, dtype: float64


In [21]:
# Load ICUSTAYS and PATIENTS tables
icustays = pd.read_csv('C:/MIMIC-III/mimic-iii-clinical-database-1.4/data/ICUSTAYS.csv')
patients = pd.read_csv('C:/MIMIC-III/mimic-iii-clinical-database-1.4/data/PATIENTS.csv')

# Merge ICUSTAYS with PATIENTS to get admission and death dates
icustays = icustays.merge(patients[['SUBJECT_ID', 'DOD']], on='SUBJECT_ID', how='left')

# Calculate time from ICU admission (INTIME) to death (DOD)
icustays['INTIME'] = pd.to_datetime(icustays['INTIME'])
icustays['DOD'] = pd.to_datetime(icustays['DOD'])
icustays['days_to_death'] = (icustays['DOD'] - icustays['INTIME']).dt.total_seconds() / (24 * 60 * 60)

# Define 30-day survival
# If days_to_death is NaN (patient survived) or > 30, they survived 30 days
icustays['surv_30d'] = ((icustays['days_to_death'].isna()) | (icustays['days_to_death'] > 30)).astype(int)

# Merge with final_data
final_data = final_data.merge(icustays[['ICUSTAY_ID', 'surv_30d']], left_on='icustay_id', right_on='ICUSTAY_ID', how='left')
final_data = final_data.drop(columns=['ICUSTAY_ID'], errors='ignore')

# Verify the outcome
print("Distribution of 30-Day ICU Survival:")
print(final_data['surv_30d'].value_counts(normalize=True))

Distribution of 30-Day ICU Survival:
surv_30d
1    0.875754
0    0.124246
Name: proportion, dtype: float64


In [22]:
print("\nMissing Value Counts and Percentages for Each Variable:")
total_rows = len(final_data)

for col in final_data.columns:
    missing_count = final_data[col].isna().sum()  # Count of missing values
    missing_percent = (missing_count / total_rows) * 100  # Percentage of missing values
    print(f"{col}: {missing_count} missing ({missing_percent:.2f}%)")

Shape of final_data: (34472, 140)

Missing Value Counts and Percentages for Each Variable:
icustay_id: 0 missing (0.00%)
subject_id: 0 missing (0.00%)
hadm_id: 0 missing (0.00%)
gender: 0 missing (0.00%)
ethnicity: 0 missing (0.00%)
age: 0 missing (0.00%)
insurance: 0 missing (0.00%)
admittime: 0 missing (0.00%)
diagnosis_at_admission: 1 missing (0.00%)
dischtime: 0 missing (0.00%)
discharge_location: 0 missing (0.00%)
fullcode_first: 6310 missing (18.30%)
dnr_first: 6310 missing (18.30%)
fullcode: 6310 missing (18.30%)
dnr: 6310 missing (18.30%)
dnr_first_charttime: 31353 missing (90.95%)
timecmo_chart: 33514 missing (97.22%)
cmo_first: 6310 missing (18.30%)
cmo_last: 6310 missing (18.30%)
cmo: 6310 missing (18.30%)
deathtime: 31122 missing (90.28%)
intime: 0 missing (0.00%)
outtime: 0 missing (0.00%)
los_icu: 0 missing (0.00%)
admission_type: 0 missing (0.00%)
first_careunit: 0 missing (0.00%)
mort_icu: 0 missing (0.00%)
mort_hosp: 0 missing (0.00%)
hospital_expire_flag: 0 missing (0

In [24]:
output_path = "C:/MIMIC-Extract/processed_final_data.csv"

try:
    final_data.to_csv(output_path, index=False)
    print(f"Data saved to {output_path}")
except NameError:
    raise Exception("final_data not found in memory. Please provide the DataFrame variable name.")

Data saved to C:/MIMIC-Extract/processed_final_data.csv
